In [41]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models

# Cihaz seçimi: GPU varsa GPU kullan, yoksa CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)


Kullanılan cihaz: cuda


In [43]:
data_dir = r"C:\Users\omerf\OneDrive\Masaüstü\garbage_classification"
batch_size = 32
num_workers = 0  # Windows/Jupyter'da 0 güvenli
epochs = 10
learning_rate = 1e-4

In [45]:
# Eğitim için veri artırma
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Doğrulama için sadece boyut ve normalize
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [47]:
# ImageFolder, her klasörü bir sınıf olarak alır
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)

# %80 train, %20 validation
train_size = int(0.8 * len(full_dataset))
val_size   = len(full_dataset) - train_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])

# Val transform'u override
val_ds.dataset.transform = val_transform

train_loader = DataLoader(train_ds, batch_size=batch_size,
                          shuffle=True, num_workers=num_workers)
val_loader   = DataLoader(val_ds, batch_size=batch_size,
                          shuffle=False, num_workers=num_workers)

print("Sınıflar:", full_dataset.classes)


Sınıflar: ['battery', 'biological', 'brown-glass', 'cardboard', 'clothes', 'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass']


In [49]:
# Önceden eğitilmiş ResNet18
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Son katmanı 12 sınıfa göre değiştir
model.fc = nn.Linear(model.fc.in_features, len(full_dataset.classes))
model = model.to(device)


In [51]:
criterion = nn.CrossEntropyLoss()  # sınıflandırma için uygun
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [53]:
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()   # önceki gradyanları sıfırla
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()         # gradyanları hesapla
        optimizer.step()        # ağırlıkları güncelle

        running_loss += loss.item() * images.size(0)

    # Doğrulama
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    epoch_loss = running_loss / len(train_loader.dataset)

    print(f"Epoch {epoch}/{epochs} - Loss: {epoch_loss:.4f} - Val Acc: {val_acc:.4f}")


Epoch 1/10 - Loss: 0.3347 - Val Acc: 0.9397
Epoch 2/10 - Loss: 0.0856 - Val Acc: 0.9610
Epoch 3/10 - Loss: 0.0410 - Val Acc: 0.9649
Epoch 4/10 - Loss: 0.0259 - Val Acc: 0.9552
Epoch 5/10 - Loss: 0.0210 - Val Acc: 0.9549
Epoch 6/10 - Loss: 0.0358 - Val Acc: 0.9568
Epoch 7/10 - Loss: 0.0270 - Val Acc: 0.9575
Epoch 8/10 - Loss: 0.0161 - Val Acc: 0.9662
Epoch 9/10 - Loss: 0.0203 - Val Acc: 0.9536
Epoch 10/10 - Loss: 0.0198 - Val Acc: 0.9488


In [55]:
torch.save(model.state_dict(), "garbage_resnet18.pth")
print("Model kaydedildi: garbage_resnet18.pth")

Model kaydedildi: garbage_resnet18.pth


In [67]:
from PIL import Image
import torchvision.transforms as transforms

# Örnek resim yolu
img_path = r"C:\Users\omerf\OneDrive\Masaüstü\bio.jpg"
image = Image.open(img_path)

# Transform uygulama (train augment değil, sadece resize+normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
image = transform(image).unsqueeze(0).to(device)  # batch dimension ekle

# Modeli yükleme
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 12)
model.load_state_dict(torch.load("garbage_resnet18.pth", map_location=device))
model.to(device)
model.eval()

# Tahmin
with torch.no_grad():
    output = model(image)
    _, pred = torch.max(output, 1)
    
classes = full_dataset.classes  # 12 sınıf listesi
print("Tahmin edilen sınıf:", classes[pred.item()])


Tahmin edilen sınıf: biological
